# Case Study

## 1. User Exerperience

In [ ]:
#pip install fuzzywuzzy

In [ ]:
#pip install --user pyinputplus

In [1]:
import pandas as pd
import random
from fuzzywuzzy import process
import pyinputplus as pyip

import config
import pandas as pd
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
import pickle

from IPython.display import IFrame

/Users/lukasbauerschmidt/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def load_hot_playlist(filename_hotsongs):
    hotsongs = pd.read_csv(filename_hotsongs)
    return hotsongs

In [ ]:
def get_user_input():
    user_input_song = input("Please provide a song: ").lower().title()
    return user_input_song

In [3]:
def transform_database(hotsong_database):
    transformed_hotsong_database = hotsong_database.applymap(lambda x: x.lower().title())
    return transformed_hotsong_database

In [17]:
def check_duplicates(user_input_song, transformed_hotsong_database, title_column, artist_column):
    if (transformed_hotsong_database[title_column] == user_input_song).sum() > 1:
        duplicates = list(transformed_hotsong_database[transformed_hotsong_database[title_column] == user_input_song][artist_column].values)
        selected_artist = pyip.inputMenu(duplicates, prompt = "There seem to be duplicated entries. Which of the following artists do you meant: \n")
        new_user_input = transformed_hotsong_database[transformed_hotsong_database[artist_column] == selected_artist][title_column].values[0]
        return new_user_input
    else:
        pass

In [12]:
'''def check_spelling_mistakes(user_input_song, transformed_hotsong_database, title_column, artist_column):
    best_match, score = process.extractOne(user_input_song, transformed_hotsong_database[title_column].values)
    if score > 80:
        artists = transformed_hotsong_database[transformed_hotsong_database[title_column] == best_match][artist_column].values
        answer = pyip.inputYesNo(prompt = f"Your entry was not accurate enough. Did you mean {best_match} by {artists}? \n Y/N: ")
        if answer.lower().title() in ["Yes", "Y"]:
            return best_match
        else:
            user_input_song = input("Please provide the song and the artist name song instead: ").lower().title()
            
    else:
        pass'''

In [7]:
def choose_random_song(transformed_hotsong_database, title_column, artist_column):
    random_hotsong_name = random.choice(transformed_hotsong_database[title_column].values)
    random_hotartist_name = transformed_hotsong_database[transformed_hotsong_database[title_column] == random_hotsong_name][artist_column].values[0]
    recommendation = print("The recommended hot song is", random_hotsong_name, "by", random_hotartist_name)
    return recommendation


In [13]:
def recommend_hot_song(filename_hotsongs, title_column, artist_column, user_input_song):
    hotsongs = load_hot_playlist(filename_hotsongs)

    transformed_hotsong_database = transform_database(hotsongs)

    #user_input_song = get_user_input()

    #while user_input_song not in list(transformed_hotsong_database[title_column]):
    #        user_input_song = input("Sorry there seems no matching song available. Please provide another song: ").lower().title()
    #user_input_song = check_spelling_mistakes(user_input_song, transformed_hotsong_database, title_column, artist_column)
    
    new_user_input = check_duplicates(user_input_song, transformed_hotsong_database, title_column, artist_column)

    recommendation = choose_random_song(transformed_hotsong_database, title_column, artist_column)

    return recommendation

In [14]:
recommend_hot_song("hot songs.csv", "songs", "artist")

Your entry was not accurate enough. Did you mean As It Was by ['Harry Styles']? 
 Y/N: 'dwafwa' is not a valid yes/no response.
Your entry was not accurate enough. Did you mean As It Was by ['Harry Styles']? 
 Y/N: 'asfwaf' is not a valid yes/no response.
Your entry was not accurate enough. Did you mean As It Was by ['Harry Styles']? 
 Y/N: The recommended hot song is Lift Me Up by Rihanna


Functionality

- Sobald Lehrzeichen oder andere Zeichen/Typos eingefügt werden sollen diese entfern werden
- Sobald ein artist eingegeben wurde, fordere den User auf, einen Song statt einem artist einzugeben
- Sofern Duplikate angezeigt werden, sollten alle angezeigt werden
- Display youtube link to the song the user inserted
- If only part of the song is displayed, also find the song
- EXTRA: display the different options of the song
- Was passiert wenn "The" davor gesetzt wird

Done:
- Rechtschreibung (transformation der eingabe in Upper + Lowercase) (DONE)
- Falls song nicht gefunden wird sollte zunächst ein error ausgeworfen werden (DONE); später soll dann eine andere Funktion geschrieben werden

In Progress:

## 2. Spotify API

### Basic Spotify API Setup

In [ ]:
#connect to spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [ ]:
#Send request to API
results = sp.search(q="Lose yourself")
sp.audio_features(results["tracks"]["items"][0]["id"] )

### Song Feature Extraction

In [ ]:
#Extracting features of songs and appending them to dataframe
list_of_songs=[]
for index in range(0,len(song["tracks"]["items"])):
    list_of_songs.append(sp.audio_features(song["tracks"]["items"][index]["uri"])[0])
df=pd.DataFrame(list_of_songs)
df=df[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

### Playlist 

In [ ]:
#Extracting the songs ids from playlist
list_of_audio_features=[]
for item in range(0,10):
    #print (tracks[item]["track"]["id"])
    list_of_audio_features.append(sp.audio_features(tracks[item]["track"]["id"])[0])
df=pd.DataFrame(list_of_audio_features)    
df=df[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

In [ ]:
#Searching a playlist
playlist = sp.user_playlist_tracks("spotify", "7beGd4yYY1qpsBv6K3clFZ",market="GB")

In [ ]:
#Extracting songs of playlist
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id,market="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

### Artists

In [ ]:
Question: how to get the playlist ID?

In [ ]:
#Extract artists from track
def get_artists_from_track(track):
    return [artist["name"] for artist in track["artists"]]

In [ ]:
#Extract artists from playlist
def get_artists_from_playlist(playlist_id):
    tracks_from_playlist = get_playlist_tracks("spotify", playlist_id)
    return list(set(artist for subset in [get_artists_from_track(track["track"]) for track in tracks_from_playlist] for artist in subset))

### Albums

In [ ]:
#Extract all tracks of one artist
def get_albums_from_artist(artist_id):
    results = sp.artist_albums(artist_id, limit = 50,country="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [ ]:
#Extract all albums ID of one artist
def get_album_ids_from_artist(artist_id):
    results = sp.artist_albums(artist_id, limit = 50)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return [track["id"] for track in tracks]

In [ ]:
#Check 
set([artist["name"] for track in coldplay_albums for artist in track["artists"]])

In [ ]:
#Extracting songs of given album
def get_track_ids_from_albums(album_ids):
    return list(set([i["id"] for j in album_ids for i in sp.album(j)["tracks"]["items"]]))

### Pipeline

In [ ]:
#setup
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [ ]:
#Identify spotify markets
markets = sp.available_markets()["markets"]

In [ ]:
#Identify genres per markets & store them in dictionary
genres_per_country = {}

for market in markets:
    genres_per_country.update({market:{}})
    for category in sp.categories(country=str(market), limit=50)["categories"]["items"]:
        genres_per_country[market].update({category["name"]: category["id"]})

In [ ]:
#Identify playlist_ids per market and per genre & store them in dictionary
from random import randint
from time import sleep

for market in markets:
    for category in genres_per_country[market]:
        try:
            genres_per_country[market].update({category: [genres_per_country[market][category], sp.category_playlists(category_id = genres_per_country[market][category], country = market, limit = 1)["playlists"]["items"][0]["id"]]})
        except:
            pass

In [ ]:
#Format: Country: Category: Category_ID, Sample Playlist_ID
genres_per_country

In [ ]:
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username, playlist_id, fields = "items.track.id, next", limit=10)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [ ]:
#Clean the playlist_ids
playlist__id_list = []
for market in markets:
    for category in genres_per_country[market]:
        playlist__id_list.append(genres_per_country[market][category][1])

In [ ]:
#Check length of unclean playist_id list
len(playlist__id_list)

In [ ]:
#Check length of unclean playist_id set -> There seems to be more than 7000 duplicates
len(set(playlist__id_list))

In [ ]:
#Create final list of playlists
final_list = list(set(playlist__id_list))

In [ ]:
#Get information of first 10 songs of each playlist
df_info = []

for playlist_id in final_list:
        try:
                df_info.extend(get_playlist_tracks("spotify", playlist_id))
        except:
                pass

In [ ]:
len(df_info)

In [ ]:
import pandas as pd
data = pd.read_csv("save.csv")
df_info = list(data["track"])

In [ ]:
from time import sleep
from random import randint
from random import uniform

feature_list = []

for track_id in df_info:
        sleep(uniform(0.1, 0.7))
        try:
                feature_list.extend(sp.audio_features(track_id["track"]["id"]))
        except:
                pass
df = pd.DataFrame(feature_list)
df = df[["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"]]
df.to_csv("final.csv")

In [ ]:
feature_list
df = pd.DataFrame(feature_list)
df = df[["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

In [ ]:
#import pandas as pd
#save = pd.DataFrame(df_info)
#save.to_csv("save.csv")

In [ ]:
def get_features(track_id):
    track_features = []
    track_features.extend(sp.audio_features(track_id))
    return track_features

In [ ]:
track_features_lst = []
for track in df_info:
    try:
        track_features_lst.extend(get_features(track["track"]["id"]))
    except:
        pass

df = pd.DataFrame(track_features_lst)
df = df[["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

## 3. Clustering

In [ ]:
def read_database(filename):
    initial_df = pd.read_csv(filename)
    columns_wo_id = initial_df.columns.drop(["id"])
    final_num = initial_df.drop_duplicates().select_dtypes(include='number')
    final_str = initial_df.drop_duplicates().select_dtypes(exclude='number')
    return final_num, final_str, columns_wo_id

In [ ]:
final_num, final_str, columns_wo_id = read_database("df_complete.csv")

### 3.1 Scaling Database

In [ ]:
def scale_num_only_database(numerical_dataframe, columns_wo_id):
    scaler = StandardScaler()
    fitted_standard_scaler = scaler.fit(numerical_dataframe)
    numerical_dataframe_scaled = scaler.transform(numerical_dataframe)
    numerical_dataframe_scaled = pd.DataFrame(numerical_dataframe_scaled, columns = columns_wo_id)
    return numerical_dataframe_scaled, fitted_standard_scaler

In [ ]:
numerical_dataframe_scaled_final, fitted_standard_scaler = scale_num_only_database(final_num, columns_wo_id)

### 3.2 Clustering Database

In [ ]:
K = range(2, 21)
inertia = []

for k in K:
    print("Training a K-Means model with {} clusters! ".format(k))
    print()
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(final_num_scaled_df)
    inertia.append(kmeans.inertia_)

%matplotlib inline

plt.figure(figsize=(16,8))
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('inertia')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Elbow Method showing the optimal k')

In [ ]:
K = range(2, 20)
silhouette = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(final_num_scaled_df)
    
    filename = "Model/kmeans_" + str(k) + ".pickle"
    with open(filename, "wb") as f:
        pickle.dump(kmeans,f)
    
    silhouette.append(silhouette_score(final_num_scaled_df, kmeans.predict(final_num_scaled_df)))


plt.figure(figsize=(16,8))
plt.plot(K, silhouette, 'bx-')
plt.xlabel('k')
plt.ylabel('silhouette score')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Silhouette Method showing the optimal k')

### 3.3 Assigning Clusters to Database

In [ ]:
def train_kmeans(numerical_scaled_dataframe, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters,
                init="k-means++",
                n_init=50,  # try with 1, 4, 8, 20, 30, 100...
                max_iter=1,
                tol=0,
                algorithm="elkan",
                random_state=1234)
    fitted_kmeans = kmeans.fit(numerical_scaled_dataframe)
    return fitted_kmeans

In [ ]:
fitted_kmeans = train_kmeans(numerical_dataframe_scaled_final, 15)

In [ ]:
def predict_and_assign_clusters(trained_kmeans, numerical_scaled_dataframe):
    clusters = trained_kmeans.predict(numerical_scaled_dataframe)
    numerical_scaled_dataframe_incl_clusters = numerical_scaled_dataframe.copy()
    numerical_scaled_dataframe_incl_clusters["cluster"] = clusters
    return numerical_scaled_dataframe_incl_clusters

In [ ]:
num_df_with_clusters = predict_and_assign_clusters(fitted_kmeans, numerical_dataframe_scaled_final)

### 3.4 Appending id features back to database

In [ ]:
def combining_dataframes(dataframe_with_clusters, dataframe_without_clusters):
    final_database = pd.concat([dataframe_with_clusters.reset_index(), dataframe_without_clusters.reset_index()], axis=1)
    final_database = final_database.drop(["index"], axis=1)
    return final_database

In [ ]:
database = combining_dataframes(num_df_with_clusters, final_str)

### 3.5 Retrieving User Song Features

In [ ]:
def get_user_input():
    user_input = input("Please provide a song: ").lower().title()
    return user_input

In [ ]:
user_input = get_user_input()

In [ ]:
def retrieve_user_song_features(user_input, fitted_scaler):
    spotify_results = sp.search(q=user_input, limit=1)
    user_input_features = sp.audio_features(spotify_results["tracks"]["items"][0]["id"])
    user_input_features_df = pd.DataFrame(user_input_features)[["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"]]
    user_input_features_df = user_input_features_df.drop("id", axis=1)
    transformed_data = fitted_scaler.transform(user_input_features_df)
    user_input_features_df = pd.DataFrame(data = transformed_data, columns = ["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","duration_ms"])
    return user_input_features_df

In [ ]:
user_song_features = retrieve_user_song_features(user_input, fitted_standard_scaler)

### 3.6 Predict Cluster of User Song

In [ ]:
def predict_cluster_of_user_song(trained_kmeans, user_input_features):
    targt_cluster = trained_kmeans.predict(user_input_features)[0]
    return targt_cluster

In [ ]:
predicted_song_cluster = predict_cluster_of_user_song(fitted_kmeans, user_song_features)

### 3.7 Recommend User a Random Song of Database

In [ ]:
def recommend_random_song_in_same_cluster(user_song_cluster, final_database):
    random_index = final_database[final_database["cluster"] == user_song_cluster].sample().index[0]
    recommended_track_id = final_database.iloc[random_index,11:].values[0]
    track_info = sp.track(recommended_track_id)
    track_name = track_info['name']
    artist_name = track_info['artists'][0]['name']
    recommendation = print("The recommended song is {} by {}".format(track_name, artist_name))
    return recommendation, recommended_track_id

In [ ]:
recommend_random_song_in_same_cluster(predicted_song_cluster, database)

In [ ]:
recommendation, recommendation_track_id = recommend_random_song_in_same_cluster(predicted_song_cluster, database)

### 3.8 Play the song 

In [ ]:
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media"
      )

In [ ]:
play_song(recommendation_track_id)

### 3.9 Recommender if inserted song is not in hot playlist

In [ ]:
def recommender_if_not_hot_songs(filename, n_clusters):
    final_num, final_str, columns_wo_id = read_database(filename)
    
    numerical_dataframe_scaled_final, fitted_standard_scaler = scale_num_only_database(final_num,columns_wo_id)

    fitted_kmeans = train_kmeans(numerical_dataframe_scaled_final, n_clusters)

    num_df_with_clusters = predict_and_assign_clusters(fitted_kmeans, numerical_dataframe_scaled_final)

    database = combining_dataframes(num_df_with_clusters, final_str)

    user_input = get_user_input()

    user_song_features = retrieve_user_song_features(user_input, fitted_standard_scaler)

    predicted_song_cluster = predict_cluster_of_user_song(fitted_kmeans, user_song_features)

    recommendation, recommendation_track_id = recommend_random_song_in_same_cluster(predicted_song_cluster, database)

    recommendation

    return play_song(recommendation_track_id)

In [ ]:
recommender_if_not_hot_songs("df_complete.csv", 15)

## 4. Putting it all together

In [16]:
def song_pre_match_checker(user_input_song, transformed_hotsong_database, title_column, artist_column):
    
    match = False

    user_input_song = user_input_song.lower().title()

    best_match, score = process.extractOne(user_input_song, transformed_hotsong_database[title_column].values)

    if score > 70:
        artists = transformed_hotsong_database[transformed_hotsong_database[title_column] == best_match][artist_column].values
        answer = pyip.inputYesNo(prompt = f"Your entry was not accurate enough. Did you mean {best_match} by {artists}? \n Y/N: ")

        if answer.lower().title() in ["Yes", "Y"]:
            match = True
            return match, best_match

        else:
            user_input_song = input("What song did you mean instead. Please insert a more accurate song title: \n").lower().title()
            match = False
            return match, best_match

    else:
        match = False
        return match, best_match

In [ ]:
def final_recommender(filename_hotsongs, title_column, artist_column, filename, n_clusters):
    user_input = get_user_input()

    match, user_input = song_pre_match_checker(user_input, filename_hotsongs, title_column, artist_column)

    if match == True:
        recommend_hot_song(filename_hotsongs, title_column, artist_column)

    if match == False:
        recommender_if_not_hot_songs(filename, n_clusters)